In [ ]:
%load_ext autoreload
%autoreload 2
    
from bmp_config import path_data,envcode2env
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from bmp_behav_proc import *

import numpy as np
from pingouin import ttest
from bmp_base import radius_cursor
from datetime import datetime
import bmp_behav_proc
fnf_fhl = pjoin(path_data,'dfcs_fixhistlen.pkl')
print(fnf_fhl)
print( str(datetime.fromtimestamp(os.stat(fnf_fhl).st_mtime)))
dfcs_fixhistlen = pd.read_pickle(fnf_fhl )
dfcs_fixhistlen['environment'] = dfcs_fixhistlen['environment'].astype(int)
df_ = dfcs_fixhistlen[['environment','subject','trials','error_pscadj_abs_Tan29']]
assert not df_.duplicated(['subject','trials']).any()

# load files with computed mixedlm results (prl* files)
from datetime import datetime
import dateutil, re
import glob
dtstart = dateutil.parser.parse('april 12 2024  01:00')

prl_path_data = pjoin(path_data, '../NIH_behav_intermed_data', 'mixedlm_prl')


prls_fnf = glob.glob(prl_path_data + '/prl_*_2025*')
prl = []
for p in prls_fnf:    
    #p=  pjoin(path_data, f'prl_{ind}.npz')
    if not os.path.exists(p):
        continue

    print(p)
    match = 1
    #match = pattern.match(os.path.basename(p) )
    if match:            
        dt = datetime.fromtimestamp(os.path.getmtime(p))
        if not dt >= dtstart:
            continue    
        print('  ',p, dt)
        prl_ = np.load(p,allow_pickle=1)['arr_0']
        print(len(prl_))
        for item in prl_:
            item['dt'] = dt
        #prl__ = [(dt,_item) for _item in prl_]
        prl += list(prl_)

print('Total len ',len(prl))


In [ ]:
# READ: extract stuff from prl (the precomputed mixed linear model fits)
import re
def cnv(s):
    v = np.nan
    if s != '':
        v = float(s)
    return v

regex = re.compile(r'C\([a-zA-Z2_]+\)\[T\.([a-zA-Z_]+)\]$')
#DEBUG = 1
DEBUG = 0
clean_summary = 1 # if True, then remove fittedvalues and resid from summary
ind = 0  # for counting only
mixmr = []
for ip,p in enumerate(prl):
#for ip,p in enumerate(prl[136:136+1]):        
    dd_ = p.copy()
    dd_['ind_in_prl'] = ip
    #del dd_['s']    
    
    varn = dd_['varn']
    s2summ = dd_['s2summary']
    #transform = dd_['transform']
    
    for (s,scov),summ in s2summ.items():                
        if summ is None:
            dd_['converged'] = False
            print('skipping due to non-convergence ',s,scov)
            continue        
        dd = {}
        dd['s'] = s
        dd['scov'] = scov

        dd.update(dd_)        
        dd = dd.copy()

        dd['transform'] = dd_['transform']

        co = dd['cocoln']
        if co == 'env':
            dd['refcond'] = 'random'
        else:
            dd['refcond'] = 'pert'
        
        del dd['s2summary'] # because we need only one
            
        tab = summ.tables[0]
        converged  = tab.loc[4,3] == 'Yes'
        converged2 = tab.loc[5,3] == 'Yes'

        fet = summ.tables[1].rename(columns={'P>|z|':'pval', 'Coef.':'coef'} )    

        fet.pval = fet.pval.apply(cnv)
        fet.coef = fet.coef.apply(cnv)
        fet['pval_full'] = summ.pvalues.to_frame()[0] # because in tables[1] precision is cut to 2 floating points
        fet['coef_full'] = summ.params.to_frame()[0] # because in tables[1] precision is cut to 2 floating points
        fet0 = fet.reset_index()        
        
        dd['fet0'] = fet0

        #fet0 = row['fet0'].set_index('index')
        
        
        def f(row):
            # Find all occurrences between [T. and ]
            s = row['index']
            match = re.findall(r"\[T\.([^\]]*)\]", s)
            if match:
                return match[0]
            else:
                return dd['refcond']
        fet0['cond'] = fet0.apply(f,1)

        d = fet[['pval','coef']].T.to_dict()
        dd['intercept_pval'] = summ.pvalues['Intercept'] #d['Intercept']['pval']
        dd['intercept_coef'] = summ.params['Intercept'] #d['Intercept']['coef']
                
        dd['varn_coef'] = summ.params[varn] #d[varn]['coef'] # main effect
        dd['varn_pval'] = summ.pvalues[varn] #d[varn]['pval']
        dd['converged']  = converged
        dd['converged2'] = converged2
        dd['s'] = s
        dd['scov'] = scov
        if clean_summary:
            summ.resid = None
            summ.fittedvalues = None
        dd['summary'] = summ

        dd['lilliefors_test_st'] = summ.lilliefors_test_st
        dd['lilliefors_test_pv'] = summ.lilliefors_test_pv
        dd['jarque_bera_test_st'] = summ.jarque_bera_test_st
        dd['jarque_bera_test_pv'] = summ.jarque_bera_test_pv
        dd['jarque_bera_test_skew'] = summ.jarque_bera_test_skew
        dd['jarque_bera_test_kurtosis'] = summ.jarque_bera_test_kurtosis
        
        dd['coefp_sig_max'] = np.nan
        dd['coefp_sig_min'] = np.nan
        if (s.find(r'*') < 0) and dd['varn_pval'] < 0.05:
            dd['coefp_sig_max'] = dd['varn_coef'] 
            dd['coefp_sig_min'] = dd['varn_coef'] 
            
        strs = fet0.query('index.str.contains(@varn) and pval <= 0.05')['cond'].values
        condssig = ','.join(strs)
        dd['condssig'] = condssig

        dd['R2_marginal'] = summ.pseudo_r2['R2_marginal']
        dd['R2_cond'] = summ.pseudo_r2['R2_conditional']

        print(varn,s,scov,converged,converged2)
        if DEBUG:
            display(summ)
        if converged and s.find(r'*') >= 0:
            interactions = fet0.query('index.str.contains(":")').index.values
            fet0int = fet0.loc[interactions]
                                    
            fet0['coefp'] = np.nan
            fet0.loc[fet0int.index,'coefp'] =fet0.loc[fet0int.index,'coef'] + dd['varn_coef']
            fet0.loc[fet0.query('index == @varn').index,'coefp'] = dd['varn_coef']            
            
            fet0int = fet0.loc[interactions] # for it to have coefp now                
            
            fet0sig = fet0.query('pval <= 0.05')
            if len(fet0sig):
                dd['coefp_sig_max'] = fet0sig['coefp'].max()
                dd['coefp_sig_min'] = fet0sig['coefp'].min()
            dd['coefp_max'] = fet0['coefp'].max()
            dd['coefp_min'] = fet0['coefp'].min()
            
            #fet0['is_nm_intercept'] = fet0tmp['index'].str.match(regex)
            fet0['is_nm_intercept'] = fet0['index'].str.match(regex)
            dd['intercept_nm_min'] = fet0.query('is_nm_intercept == True').coef.min()
            dd['intercept_nm_max'] = fet0.query('is_nm_intercept == True').coef.max()
            dd['intercept_nm_pval_min'] = fet0.query('is_nm_intercept == True').pval.min()
            dd['intercept_nm_pval_max'] = fet0.query('is_nm_intercept == True').pval.max()

            rs = fet0int.query('pval <= 0.05')
            lsig = len( rs )            

            num_sig_inter = lsig + int( cnv(d[varn]['pval']) <= 0.05 ) # number of coefs that are signif
            num_sig_inter_nm = lsig  # number of corrections that are significant
                
            num_sig_inter_pos = int( dd['varn_coef'] > 0 ) * int( dd['varn_pval'] < 0.05 )
            num_inter_pos = int( dd['varn_coef'] > 0 ) 
            for ir,r in rs.iterrows():
                coef_full = d[varn]['coef'] + float(r['coef'])
                num_sig_inter_pos += int(coef_full > 0)
            dd['coefp_nm_max'] = fet0int['coefp'].max()  # nm means not meain effect
            dd['coefp_nm_min'] = fet0int['coefp'].min()
            
            dd['num_inter'] = len(interactions) + 1                        
            dd['num_sig_inter'] = num_sig_inter
            dd['num_sig_inter_pos'] = num_sig_inter_pos 

            dd['num_sig_inter_nm'] = num_sig_inter_nm
        
            if scov == "1" and DEBUG:
                raise ValueError('st')        
#        display(summ.wmess)
        del summ,fet,fet0,tab        

        mixmr += [dd.copy()]
        ind += 1
    #break
assert len(mixmr)
mixmr= pd.DataFrame(mixmr)
print('completed')

mixmr['inter'] = mixmr['s'].str.contains(r'\*') # contains treats string as a regular expression
mixmr['sshrt'] = mixmr['s'].str[10:]

def f(row):
    varn = row['varn']
    s = row['sshrt']
    if isinstance(s,str):
        s = s.replace(varn,'{varn}')
    return s
mixmr['sshrt'] = mixmr.apply(f,1)

print(len(prl),len(mixmr))

mixmr.loc[mixmr['num_inter'].isna(),'num_inter'] = 0
mixmr = mixmr.loc[~mixmr.s.str.contains(r'\* error_pscadj_abs')] # they are weird, because not prev

# take newer calcs
grp = mixmr.groupby(['cocoln','varn','sshrt','scov','transform'])
mixmr = aggRows(mixmr,'dt','max', grp =grp )

def f(row):
    statn = row['varn'][len(row['varn0'])+1:]
    hl = row['histlen']
    hl_l = len(str(hl))
    statn = statn[:-hl_l]
    return statn
                  
mixmr['statn'] = mixmr.apply(f,1)

In [ ]:
#s2summ

In [ ]:
load = 1
save = 0
if load:
    #date = ''
    mll_fn = 'mll_2025-07-20_00:16:50'
    mixmr = pd.read_pickle(pjoin(prl_path_data,f'{mll_fn}.pkl.zip') )
else:
    if save:    
        s = str(datetime.now())[:-7].replace(' ','_')
        mixmr.to_pickle(pjoin(prl_path_data, f'mll_{s}.pkl.zip'), compression='zip')
        print('saved')

# after collection (assuming mixmr is ready to use)

In [ ]:
# sanity check that we don't have repeats
sz = mixmr.groupby(['cocoln','varn','sshrt','scov','transform']).size()
print(sz.max())
#display(sz)
assert sz.max() == 1

In [ ]:
# load the best model (in R2 sense) that has significant pvalue
inds = mixmr.query('varn_pval <= 0.05 and cocoln == "env"').\
    sort_values(['R2_cond'], ascending=False).head(1).index._data
#inds = inds[2:]
#[cols_fav]
cc = ['Coef.','P>|z|']
for ind in inds:
    row = mixmr.loc[ind]
    print(f'index = {ind} (indprl ={row["ind_in_prl"]}), formula = {row["s"]},'
          f'\n    cov structure = {row["scov"]}')
    print('transform = ', row['transform'])
    print(f"R2 conditional = {row['R2_cond']:.4f}, R2 marginal = {row['R2_marginal']:.4f}")
    print(f"lilliefors_test_pv: {row['lilliefors_test_pv']:.4f}")
    print(f"jarque_bera_test_pv: {row['jarque_bera_test_pv']:.4f}")
    print(f"jarque_bera_test_skew: {row['jarque_bera_test_skew']:.4f}")
    print(f"jarque_bera_test_kurtosis: {row['jarque_bera_test_kurtosis']:.4f}")
    fet = row['summary'].tables[1]
    #rowis = fet.index[:-3] # .iloc[:-2]
    print(fet[cc])
    print('--------------')
    #break
varnames = list( mixmr.loc[inds]['varn'].values )
print(varnames)

In [ ]:
# plot QQ plot
%matplotlib inline
from statsmodels.stats.diagnostic import het_breuschpagan
import statsmodels.api as sm
ind_in_prl = mixmr.loc[inds[0],'ind_in_prl']
prl_el = prl[ind_in_prl]
for k,summ in prl_el['s2summary'].items():
    if summ is None:
        continue
    if summ.resid is None:
        continue
    print(k)
    sm.qqplot(summ.resid, line='s')
    plt.title(f'QQ Plot for\n {k}, \ntransform: {prl_el["transform"]}')
    plt.show()

    sns.scatterplot(x=summ.fitted_values, y=summ.resid, alpha=0.5)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("Fitted Values")
    plt.ylabel("Residuals")
    plt.title(f'Residuals vs. Fitted Values\n {s}\ntransform: {prl_el["transform"]}')
    plt.show()

# Before starting ot care about quality of fits

In [ ]:
#cols_exc = ['histlen','varn0','res','summary','varn_suffix', 's2summary','s']
cols_exc = ['histlen','varn0','res','summary','varn_suffix', 's2summary','converged',
            'converged2','s', 'excfmt','fet0']
cols0 = [c for c in mixmr.columns if c not in cols_exc]; cols0
#display(mixmr[cols])

cols_fav = ['transform', 'cocoln', 'varn', 'scov', 
 'coefp_sig_max', 'coefp_sig_min', 'coefp_nm_max', 'coefp_nm_min', 
    'num_sig_inter_pos','sshrt','condssig','R2_marginal','R2_cond', 
    'lilliefors_test_pv','jarque_bera_test_pv','jarque_bera_test_skew',
    'jarque_bera_test_kurtosis'] #'num_inter',

qs = 'cocoln == "env" and varn_pval <= 0.05 and converged2 == True and excfmt.isnull()'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
    
#qs += ' and coefp_sig_max > 0'
#display(mixmr.sort_values('s').query(qs)[cols0])
display(mixmr.sort_values(['s']).query(qs)[cols_fav])

### All positive slopes

In [ ]:
# with significant interactions with ALL positive slopes (so the most inconsistent slopes across conditions)
# there is only one for ps_ error_pscadj_abs_invstd7 and the slope size is pretty low
cols = [c for c in mixmr.columns if c not in cols_exc]; cols

qs = 'converged2 == True'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
    
qs += ' and coefp_sig_max > 0 and inter == True and num_sig_inter_pos == num_inter'
ds_ = mixmr.sort_values(['inter','varn']).query(qs)
inds = ds_.index._data
display(ds_[cols])

ds_2 = mixmr.sort_values(['inter','varn']).query(qs + ' and scov != "1"')
inds2 = ds_2.index._data
print('and with random effects inds = ', inds)

In [ ]:
# 252 has pretty small slopes in pres and rnd, but large in rnd and pert
cc = ['Coef.','P>|z|']
for ind in inds:
    row = mixmr.loc[ind]
    print(ind, row['s'], row['scov'])    
    fet = row['summary'].tables[1]
    #rowis = fet.index[:-3] # .iloc[:-2]
    print(fet[cc])
    #break
varnames = list( mixmr.loc[inds]['varn'].values )
print(varnames)

In [ ]:
# plot relationship
h=4
me = dfcs_fixhistlen.\
    groupby(['subject','env'])[ ['err_sens']+varnames].mean().reset_index()
fg,df = relplot_multi(sep_ys_by = 'col', data=me, x='err_sens',ys=varnames, row='env',
                     height = h)
fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Mean within subject')

fg,df = relplot_multi(sep_ys_by = 'col', data=dfcs_fixhistlen, x='err_sens',ys=varnames, row='env',
                     height = h)
fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Pooled')

In [ ]:
# with at least 1 condition signif different (so with slopes not fully consistent)
cols = [c for c in mixmr.columns if c not in cols_exc + ['varn_pval'] ]; cols

qs = 'converged2 == True'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
qs += ' and coefp_sig_max > 0 and inter == True and num_sig_inter > 1 and coefp_sig_max >= 0.01'
qs += ' and not s.str.contains("error_pscadj_abs")'
df_ = mixmr.sort_values(['cocoln','inter','varn']).query(qs)
inds = df_.index._data
#display(df_[cols])

df_[cols_fav]

### some positive slopes

In [ ]:
# with >= 1 significant slopes
# there is only one for ps_ error_pscadj_abs_invstd7 and the slope size is pretty low
cols = [c for c in mixmr.columns if c not in cols_exc]; cols

qs = 'converged2 == True'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
    
qs += ' and coefp_sig_max > 0 and inter == True and not s.str.contains("prev_")' 
ds_ = mixmr.sort_values(['inter','varn']).query(qs)
inds = ds_.index._data
display(ds_[cols])

ds_2 = mixmr.sort_values(['inter','varn']).query(qs + ' and scov != "1"')
inds2 = ds_2.index._data
print(f'and with random effects {len(inds2)} inds2 = ', inds)

In [ ]:
#[c for c in dfc.columns if c.find('error') >= 0]

In [ ]:
#mixmr.iloc[0]['s2summary'].keys()

In [ ]:
ds_2[cols_fav ]

In [ ]:
# the first one (with random effects for ps_) is error_pscadj_invstd19, it has rahter small slope (0.022)
# and it has a tendency (not signif) to decrease in random (-0.048)

# other indices (w/o random effects for env) are for std with high histlen 25+, 
# they have very high slope in random with strong (not signif) reduction in stable
# but it goes doown
cc = ['Coef.','P>|z|']
for ind in inds2:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['converged2'])    
    #print(row['s'], '---  ',row['scov'])    
    #display(row['summary'].wmess)
    print(row['varn'])#, '---  ',row['scov'])    
    fet0 = row['fet0'][['index','cond','coef','coefp','pval']]#.tables[1]
    #rowis = fet.index[:-3] # .iloc[:-2]
    #display(fet[cc])
    display(fet0)
    #break

###  with just one signif condition (so with slopes kinda consistent across conditions)

In [ ]:
# with just one signif condition (so with slopes kinda consistent across conditions)
cols = [c for c in mixmr.columns if c not in cols_exc + ['varn_pval'] ]; cols

qs = 'converged2 == True'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
qs += ' and coefp_sig_max > 0 and inter == True and num_sig_inter == 1 '#'and coefp_sig_max >= 0.01'
qs += ' and not s.str.contains("error_pscadj_abs")'
df_ = mixmr.sort_values(['cocoln','inter','varn0','histlen']).query(qs)
inds = df_.index._data
print(len(inds))
varnames = mixmr.loc[inds,'varn'].values
#display(df_[cols])

#df_2 = mixmr.sort_values(['cocoln','inter','varn']).query(qs + ' and scov != "1"')
#inds = ds_2.index._data
#print(inds)

In [ ]:
#print(set(varnames) )
from collections import OrderedDict
#my_list = [1, 2, 3, 2, 5, 1]
list(OrderedDict.fromkeys(varnames))

In [ ]:
#print(cols)
df_[cols_fav] #,'intercept_pval', 'intercept_coef',num_sig_inter

In [ ]:
# the first one (with random effects for ps_) is error_pscadj_invstd19, it has rahter small slope (0.022)
# and it has a tendency (not signif) to decrease in random (-0.048)

# other indices (w/o random effects for env) are for std with high histlen 25+, 
# they have very high slope in random with strong (not signif) reduction in stable
# but it goes doown
cc = ['Coef.','P>|z|']
for ind in inds:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['converged2'])    
    #print(row['s'], '---  ',row['scov'])    
    display(row['summary'].wmess)
    print(row['varn'])#, '---  ',row['scov'])    
    fet = row['summary'].tables[1]
    #rowis = fet.index[:-3] # .iloc[:-2]
    #display(fet[cc])
    display(fet)
    #break

In [ ]:
mixmr.varn.unique()

In [ ]:
from figure.plots import relplot_multi

In [ ]:
# scatter plots don't show a very clear relationship between std for high histlen and err_sens
fg,df__ = relplot_multi(sep_ys_by = 'row', data=dfcs_fixhistlen,x='err_sens',ys=varnames, col='env')
             #row='subject');
fg.refline(x=0)
fg.refline(y=0)

In [ ]:
# histograms of error_std for large histlen show that they have fat tail towards zero for many participants
dfcs_fixhistlen.query('subject_ind == 13').hist(varnames[-1])
dfcs_fixhistlen.hist(varnames[-1])

In [ ]:
display(mixmr.query('varn == @varnames[-1]')[cols0])
display(mixmr.loc[39].summary.tables[1])

### --- with more restriction on coefp

In [ ]:
# --- with more restriction on coefp
qs = 'converged2 == True'
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
qs += ' and coefp_sig_max > 0 and inter == True and num_sig_inter == 1'# and coefp_sig_max >= 0.1'
qs += ' and histlen >= 4'
#qs += ' and coefp_min > -0.3'
df_ = mixmr.sort_values(['cocoln','varn0','histlen','scov']).query(qs)
inds = df_.index.values

df_[cols_fav] #,'intercept_pval', 'intercept_coef',num_sig_inter

In [ ]:

#cc = ['Coef.','P>|z|']
for ind in inds:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['coefp_sig_min'])    
    fet0 = row['fet0']
    display(fet0.iloc[:-1][['index','coef','coefp','pval']])
    #rowis = fet.index[:-3] # .iloc[:-2]
    #display(fet[cc])
    #break
print(len(inds))

In [ ]:
# for small histlen (up to 4 inc)  std has large slope in stable (and small/moderate insignif diff in random)
# for large histlen (since 25) std has large slope for random (and moderate/large insignif diff in stable )

In [ ]:
varnames = mixmr.loc[inds,'varn'].values
print(varnames, set(varnames))
#varnames_toshow = #[varnames[0]] + [varnames[-1] ]

In [ ]:
relplot_multi(data=dfcs_fixhistlen,x='err_sens',ys=varnames_toshow[:1], col='env');
relplot_multi(data=dfcs_fixhistlen,x='err_sens',ys=varnames_toshow[1:], col='env');
             #row='subject');
#sns.relplot(kind='scatter',data=dfcs_fixhistlen, x=

### No inter env (for Tan comparison)

In [ ]:
# no inter
# for env all intercepts are significant
# for ps_ all non-random intercepts are significant
qs = 'varn_pval <= 0.05 and converged2 == True '
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
qs += ' and coefp_sig_max > 0 and inter == False and scov != "1" and cocoln == "env"'
# we exclude std3 because it is too close to std2 which is algebraically close to err sens formula
qs += ' and not s.str.contains("error_pscadj_abs") and varn != "error_pscadj_std3"'
#qs += ' and coefp_sig_max >= 1e-3'
df_ = mixmr.sort_values(['cocoln','inter','varn0','histlen']).query(qs)
inds = df_.index._data
#display(df_[cols_fav[:4] + ['intercept_coef','intercept_pval']])
display(df_[cols_fav[:4] + ['R2_cond','R2_marginal','intercept_coef','intercept_pval']])
print(len(inds))

varn_suffixes_nointer = df_['varn_suffix'].unique()
print(varn_suffixes_nointer)

In [ ]:
df_srt = df_.sort_values('varn_coef', ascending=False)[cols_fav + ['varn_pval']]
display(df_srt)

In [ ]:
#df_.iloc[0]['fet0']

In [ ]:
# TODO: apply to env for stable
def addcols(df, cocoln, cond, pref, prek_pattern):
    '''pref determines names of the output columns'''
    #prek = 'C(ps2_)[T.washout]'
    prek = prek_pattern.format(cocoln,cond)
    #prek = f'C({cocoln})[T.{cond}]'
    def f(row):
        #fet0 = row['fet0']
        varn_cur = row['varn']
        #k = prek + f':{varn_cur}'
        ps,pvs = row['summary'].params, row['summary'].pvalues
        
        k = prek
        if k not in ps:
            return None,None
        coef = ps[k]
        pv   = pvs[k]            
        icpt,icpt_pv = ps[k], pvs[k]

        return icpt,icpt_pv
    cols = []
    for s in ['icpt','icptpv']:
        cols += [pref + s]
    df[cols] = df.apply(f,1,result_type='expand')
    return cols
addcols(df_, 'env', 'stable', 's_', 'C({})[T.{}]')    
addcols(df_, 'env', 'stable', 'r_', 'Intercept')    
df_[cols_fav + ['s_icpt','s_icptpv','r_icpt','r_icptpv']]

In [ ]:
#row.keys()

In [ ]:
br = df_.query('r_icptpv < 0.05')#
#print('random intercept is signif')
#display(br[cols_fav + ['s_icpt','s_icptpv','r_icpt','r_icptpv']])
hlsig2,s_icpt2,s_icptpv2,r_icpt2,r_icptpv2 =\
    br.iloc[1][['histlen'] + ['s_icpt','s_icptpv','r_icpt','r_icptpv'] ]

br = df_.query('s_icptpv < 0.05 and r_icptpv < 0.05')#
print('When both intercepts are signif and different')
display(br[cols_fav + ['s_icpt','s_icptpv','r_icpt','r_icptpv']])
_,s_icpt,s_icptpv,r_icpt,r_icptpv =\
    br.iloc[0][['histlen'] + ['s_icpt','s_icptpv','r_icpt','r_icptpv'] ]
histlens_both_signif = br['histlen'].values
hlsig = histlens_both_signif
hlsig = ', '.join( map(str, hlsig ) )
print(histlens_both_signif)

rows = br.iloc[[0,-1]]
s0_icpt = ('With growing history length the slope estimate grows, while random and stable' 
      ' intercept values decrease: ')
for rowi,row in rows.iterrows():
# h1,h2 = histlens_both_signif[[0,-1]]
# c1,c2 = print(br['coefp_sig_max'].values[[0,-1]])
# r1,r2 = print(br['r_icpt'].values[[0,-1]])
# s1,s2 = print(br['s_icpt'].values[[0,-1]])
    s = ('for history length {}: slope {:.2f} (p-value = {:.2e}), ' 
    'random intercept = {:.2e} (p-value = {:.2e})'
    'stable intercept correction = {:.2e} (p-value = {:.2e}). ')\
        .format(row['histlen'], row['varn_coef'], row['varn_pval'], row['r_icpt'], row['r_icptpv'],
               row['s_icpt'],row['s_icptpv'])
    s0_icpt += s
print(s0_icpt)




In [ ]:
br = df_.nlargest(1,'varn_coef')
display(br[cols_fav + ['ind_in_prl']])#[['varn_coef','varn_pval']]
row = br.iloc[0]; print(row.name)
pv,coef, hlmax = row[['varn_pval','varn_coef','histlen']].values

br = df_.nsmallest(1,'varn_coef')
display(br[cols_fav + ['ind_in_prl']])#[['varn_coef','varn_pval']]
row = br.iloc[0]; print(row.name)
pv_min,coef_min, hlmin = row[['varn_pval','varn_coef','histlen']].values

In [ ]:
dfu = df_.groupby(['cocoln','statn'])['histlen'].unique()
print( dfu  )
lens =  dfu['env']['invstd'] 
mnl,mxl,diffl = min(lens),max(lens), np.diff(lens)
print(mnl,mxl,diffl)
lens = ', '.join( map(str, lens ) )
#between {mnl} and {mxl}. 
s = f'''We ran mixed linear models ES ~ error statistics
with two conditions (random,stable)
without interactions but with random effects. Here "error statistics" is one of 
many possible statistics of previous errors: mean, standard deviation, inverse standard deviation,
mean/standard deviation, mean squared/variance.
We selected those models that converged and have a significant positive slope.
We found that only models with 1 / standard deviation with history lengths {lens}.
In by publication by Tan at al [CITE] authors used mean squared/variance with history lengths 20. 
However for our data this statistics appears to be not the right one.
The maximum histlen we tried was 39 so it is possible that for longer lengths same holds, 
probably because standard deviation stops changing much. 
History length higher than 6 was increased in 3-trial step.
The largest slope {coef:.3f} (p-value = {pv:.2e}) was for history of length {hlmax}.
The smallest slope {coef_min:.3f} (p-value = {pv_min:.2e}) was for history of length {hlmin}.
The only history lengths for which intercepts for both conditions were significantly 
positive and different was {hlsig}.'''
s += s0_icpt
s += '''In all other cases intercepts were not significantly different from zero.
'''.replace('\n',' ').replace('  ',' ')
print('\n'+s)

# (intercept for random = {r_icpt:.3f}, it is > 0 
# with p-value {r_icptpv:.2e} and for stable it's difference from random was
# {s_icpt:.3f}, it is > 0 with p-value {s_icptpv:.2e}). 
# In addition for history length {hlsig2} the intercept {r_icpt:.3f} for random > 0 was significant 
# with p-value {r_icptpv:.2e}.

### No inter ps2_  (reported in paper as of  07.07.24)

In [ ]:
#dfall.query('subj == "01" and ps2_ == "washout" and block_name == "stable1"')

In [ ]:
# no inter
# for env all intercepts are significant
# for ps_ all non-random intercepts are significant
qs = 'varn_pval <= 0.05 and converged2 == True '
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
qs += ' and coefp_sig_max > 0 and inter == False and scov != "1" and cocoln == "ps2_"'
# we exclude std3 because it is too close to std2 which is algebraically close to err sens formula
qs += ' and not s.str.contains("error_pscadj_abs") and varn != "error_pscadj_std3"'
#qs += ' and histlen <= 24'
#qs += ' and coefp_sig_max >= 1e-05'
df_ = mixmr.sort_values(['cocoln','inter','varn0','histlen']).query(qs)
inds = df_.index._data
#display(df_[cols_fav[:4] + ['intercept_coef','intercept_pval']])
display(df_[cols_fav[:4] + ['R2_cond','R2_marginal','intercept_coef','intercept_pval']])
print(len(inds))

varn_suffixes_nointer = df_['varn_suffix'].unique()
print(varn_suffixes_nointer)

In [ ]:
df_srt = df_.sort_values('varn_coef', ascending=False)[cols_fav + ['varn_pval']]
display(df_srt)

In [ ]:
br = df_.query('cocoln == "ps2_"').nlargest(1,'varn_coef')
display(br[cols_fav + ['ind_in_prl']])#[['varn_coef','varn_pval']]
row = br.iloc[0]
print(row.name)
pv,coef, hlmax = row['varn_pval'],row['varn_coef'],row['histlen']

In [ ]:
iceptrows = ['Intercept','C(ps2_)[T.pre]','C(ps2_)[T.washout]','C(ps2_)[T.rnd]']
display(row['fet0'].set_index('index').loc[iceptrows,['cond','coef','pval']] )
display(row['summary'].pvalues.to_frame().loc[iceptrows])

For this statistic the intercept at baseline condition (stable perturbation) was not statistically different from zero. Corrections to it at washout and no perturbation conditions were significantly positive (coefficients 0.75 and 0.31 with p-values 9.25e-26 and 1.19e-10)

In [ ]:
dfu = df_.groupby(['cocoln','statn'])['histlen'].unique()
print( dfu  )

lens = dfu['ps2_']['invstd']
mnl,mxl,diffl = min(lens),max(lens), np.diff(lens)
lens = ', '.join(map(str,lens))
print(mnl,mxl,diffl)

#between {mnl} and {mxl}. 
s = f'''We also ran mixed linear models for
four conditions (random, zero perturbation, perturbation, washout)
without interactions but with random effects. 
We selected those models that converged and have a significant positive slope.
Similar to the case with two conditions above, 
we found that only models with 1 / standard deviation with historly lengths {lens} 
satisfy these requirements.
This time the maximum histlen we used was 24 because it is the lenght of the shortest condition.
The largest slope {coef:.3f} (p-value = {pv:.2e}) was for history of length {hlmax}.
'''.replace('\n',' ').replace('  ',' ')
print('\n'+s)

In [ ]:
#for ind in inds:
for ind in [4699]:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['coefp_sig_min'])    
    print(row['varn'])

    display(mixmr.loc[ind].summary.tables[1][['Coef.','P>|z|']])

### some slopes WITH interaction, only those stats that work for no interactions (reported in paper as of 07.07.24)

In [ ]:
# with >= 1 significant slopes
# there is only one for ps_ error_pscadj_abs_invstd7 and the slope size is pretty low
cols = [c for c in mixmr.columns if c not in cols_exc]; cols

qs = 'converged2 == True and cocoln == "ps2_" '
qs += ' and coefp_sig_max > 0 and inter == True and not s.str.contains("prev_")' 
qs += ' and varn_suffix.isin(@varn_suffixes_nointer)'
# choose those that have more than one signif interaction (not necessarily positive)
qs += ' and condssig.str.contains(",")'  
qs += ' and histlen <= 24'
qs += ' and varn0 == "error_pscadj"'

df_ = mixmr.sort_values(['cocoln','inter','varn_suffix','histlen']).query(qs + ' and scov != "1"')

display(df_[cols_fav])
inds = df_.index._data
print(f'and with random effects {len(inds)} inds = ', inds)

dftmp = df_.copy()
dftmp = dftmp.set_index('histlen', verify_integrity=True)

print( dftmp.index.values )

In [ ]:
# TODO: apply to env for stable
def addcols(df, cocoln, cond, pref):
    '''pref determines names of the output columns'''
    #prek = 'C(ps2_)[T.washout]'
    prek = f'C({cocoln})[T.{cond}]'
    def f(row):
        #fet0 = row['fet0']
        varn_cur = row['varn']
        k = prek + f':{varn_cur}'
        ps,pvs = row['summary'].params, row['summary'].pvalues
        if k not in ps:
            return None,None,None,None
        coef = ps[k]
        pv   = pvs[k]    

        k = prek
        icpt,icpt_pv = ps[k], pvs[k]

        return coef,pv,icpt,icpt_pv
    cols = []
    for s in ['coef','pv','icpt','icptpv']:
        cols += [pref + s]
    df[cols] = df.apply(f,1,result_type='expand')
    return cols
addcols(dftmp, 'ps2_', 'washout', 'w')    
dftmp[['varn','R2_cond','varn_coef','varn_pval','intercept_coef','intercept_pval'] + ['wcoef','wpv','wicpt','wicptpv']]

In [ ]:
rows = dftmp.reset_index().iloc[[0,-1]]
s0_slope = ('''with growing history length the slope estimate in perturbation grows and
the postive significant correction to it in washout as well
while random and stable intercept values decrease: ''').replace('\n',' ')
for rowi,row in rows.iterrows():
# h1,h2 = histlens_both_signif[[0,-1]]
# c1,c2 = print(br['coefp_sig_max'].values[[0,-1]])
# r1,r2 = print(br['r_icpt'].values[[0,-1]])
# s1,s2 = print(br['s_icpt'].values[[0,-1]])
    s = ('for history length {}: ' 
    'perturbation slope = {:.2e} (p-value = {:.2e}) '
    'washout slope correction = {:.2e} (p-value = {:.2e}). ')\
        .format(row['histlen'], row['varn_coef'], row['varn_pval'], row['wcoef'], row['wpv'])
    s0_slope += s
print(s0_slope)

In [ ]:
coef1,pv1 = dftmp.loc[12][['wcoef','wpv']].values
coef2,pv2 = dftmp.loc[15][['wcoef','wpv']].values

refcond = 'pert'
othercond = 'washout'
lens = ', '.join(map(str,df_.histlen.values))
s = f'''To check whether the slope changes across conditions, 
for those statistical measures we found above we also computed mixed linear model with interactions 
and random effects. We found that only two models had significant slope difference between some conditins:
for models with with history lengths = {lens}
where in {othercond} condition slopes were significantly different.
For all the models the intercept in this condition was not significantly different from the 
reference condition ({refcond}) but the slope was significantly higher than in reference condition. 
''' 
s = s.replace('\n',' ').replace('  ',' ') + '\n\nMore specifically, ' + s0_slope
s += ' This is consistent with naive visual inspection of Fig 2A'
s += ' suggesting that adaptation is washout is faster than in perturbation stage.'
print(s)

# For len 12: slope correction {coef1:.3f}, p-value = {pv1:.2e}, 
# for len 15: slope correction {coef2:.3f}, p-value {pv2:.2e}.

In [ ]:
cc = ['Coef.','P>|z|']
iceptrows = ['Intercept','C(ps2_)[T.pre]','C(ps2_)[T.washout]','C(ps2_)[T.rnd]']
fet_icepts = []
for ind in inds:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['converged2'])    
    #print(row['s'], '---  ',row['scov'])    
    #display(row['summary'].wmess)
    print(row['varn'])#, '---  ',row['scov'])    
    fet0 = row['fet0'][['index','cond','coef','coefp','pval']]#.tables[1]
    #rowis = fet.index[:-3] # .iloc[:-2]
    #display(fet[cc])
    #display(fet0)
    fet_icept = row['fet0'].set_index('index').loc[iceptrows,['cond','coef','pval_full']]
    fet_icept['histlen'] = row['histlen']
    fet_icepts += [fet_icept]
    display( fet_icept )
fet_icepts = pd.concat(fet_icepts)
fet_icepts = fet_icepts.reset_index()    

In [ ]:
#fet_icepts = fet_icepts.reset_index()    
fet_icepts.groupby(['index','cond'])[['coef','pval_full']].describe(percentiles=[])

### Table for supp

In [ ]:
# v1, where I take those when in ANY condition one coef was signif and pos
qs = 'converged2 == True '
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
#    and coefp_sig_max > 0 and 

# with interactions but with simple covariance
qs += ' and inter == True and scov != "1" and cocoln == "ps2_"'
# we exclude std3 because it is too close to std2 which is algebraically close to err sens formula
qs += ' and not s.str.contains("error_pscadj_abs") and varn != "error_pscadj_std3"'
qs += ' and histlen <= 24'
qs += ' and coefp_sig_max >= 1e-05'
#qs += ' and coefp_min >= 0'   # all positive
df_ = mixmr.sort_values(['cocoln','inter','varn0','varn_suffix','histlen']).query(qs)

df_['all_slopes_pos'] = (df_['coefp_min'] >= 0) # smallest correcte coef is non-neg
df_['all_slopes_equal'] = (df_['num_sig_inter_nm'] == 0) # all non-main interations are non-significant
df_['all_intercepts_equal'] = (df_['intercept_nm_pval_max'] >= 0.05) # all non-main intercepts are non-significant

from bmp_behav_proc import formatRecentStatVarnames
df_['stat_name_nice'] = df_['varn_suffix'].apply(lambda x: formatRecentStatVarnames(['_'+x], histlen_str = '')[0][1:] )
print(len(df_))

In [ ]:
# v2, where I take those when in main effect was signif and pos
qs = 'converged2 == True '
# with pd.option_context('display.max_rows', None):
#     display(mixmr.sort_values('s').query(qs)[cols])
#    and coefp_sig_max > 0 and 

# with interactions but with simple covariance
qs += ' and inter == True and scov != "1" and cocoln == "ps2_"'
# we exclude std3 because it is too close to std2 which is algebraically close to err sens formula
qs += ' and not s.str.contains("error_pscadj_abs") and varn != "error_pscadj_std3"'
qs += ' and histlen <= 24'
qs += ' and varn_coef >= 1e-05 and varn_pval <= 0.05'
#qs += ' and coefp_min >= 0'   # all positive
df_ = mixmr.sort_values(['cocoln','inter','varn0','varn_suffix','histlen']).query(qs)

df_['all_slopes_pos'] = (df_['coefp_min'] >= 0) # smallest correcte coef is non-neg
df_['all_slopes_equal'] = (df_['num_sig_inter_nm'] == 0) # all non-main interations are non-significant
df_['all_intercepts_equal'] = (df_['intercept_nm_pval_max'] >= 0.05) # all non-main intercepts are non-significant

from bmp_behav_proc import formatRecentStatVarnames
df_['stat_name_nice'] = df_['varn_suffix'].apply(lambda x: formatRecentStatVarnames(['_'+x], histlen_str = '')[0][1:] )
print(len(df_))

In [ ]:
df_['coefp_sig_range'] = df_['coefp_sig_max'] - df_['coefp_sig_min']
df_['coefp_sig_range_frac'] = df_['coefp_sig_range'] / df_['coefp_sig_max']

In [ ]:
#- whether all slopes are positive
#- whether all slopes are equal,
#- whether all intercepts are equal, whether
cols_R = ['cocoln','histlen','varn0','varn_suffix','num_sig_inter',
          'sshrt','all_slopes_pos','all_slopes_equal','all_intercepts_equal']
df_[cols_R]

In [ ]:
df_.query('varn0 == "error_pscadj" and varn_suffix == "invstd"')[['histlen','varn_coef','coefp_min','coefp_max','coefp_sig_max']]#.coefp_max#.max()

In [ ]:
cols_Rf = ['histlen','stat_name_nice','all_slopes_pos','all_slopes_equal','all_intercepts_equal', 
           'coefp_sig_range_frac_pct_nice','R2_marginal_pct','R2_cond_pct']
df_['R2_marginal_pct'] = df_['R2_marginal'] * 100
df_['R2_cond_pct'] = df_['R2_cond'] * 100
dfres0 = df_.query('varn0 == "error_pscadj"').copy()
dfres0['coefp_sig_range_frac_pct'] = dfres0['coefp_sig_range_frac'] * 100
dfres0['coefp_sig_range_frac_pct_nice'] = dfres0['coefp_sig_range_frac_pct'].apply(lambda x: f'{x:.0f}%')
dfres = dfres0[cols_Rf].rename(columns={'histlen':'history length', 
        'stat_name_nice':'window statistic formula', 'all_slopes_pos':'all slopes positive',
        'all_slopes_equal':'all slopes equal','all_intercepts_equal':'all intercepts equal',
                                       'coefp_sig_range_frac_pct_nice':'max slope change'})
with pd.option_context('display.precision', 1):
    display(dfres)
dfres.to_csv(pjoin(path_fig, 'behav', 'Tanlike_statistics_summary_table.csv'), index=False)

In [ ]:
# detailed
inds = df_.query('varn0 == "error_pscadj" and histlen.isin([17,18,19,20,21,22])').sort_values('histlen').index._data
#inds = df_.query('varn0 == "error_pscadj" and histlen.isin([14])').sort_values('histlen').index._data
inds = df_.query('varn0 == "error_pscadj" and histlen.isin([7])').sort_values('histlen').index._data

cc = ['Coef.','P>|z|']
iceptrows = ['Intercept','C(ps2_)[T.pre]','C(ps2_)[T.washout]','C(ps2_)[T.rnd]']
fet_icepts = []
for ind in inds:
    row = mixmr.loc[ind]
    print(ind, row['s'], '---  ',row['scov'], row['converged2'])    
    #print(row['s'], '---  ',row['scov'])    
    #display(row['summary'].wmess)
    print(row['varn'])#, '---  ',row['scov'])    
    fet0 = row['fet0'][['index','cond','coef','coefp','pval']]#.tables[1]
    #rowis = fet.index[:-3] # .iloc[:-2]
    #display(fet[cc])
    display(fet0)
    fet_icept = row['fet0'].set_index('index').loc[iceptrows,['cond','coef','pval_full']]
    fet_icept['histlen'] = row['histlen']
    fet_icepts += [fet_icept]
    display( fet_icept )
fet_icepts = pd.concat(fet_icepts)
fet_icepts = fet_icepts.reset_index()    

## corr for all without regard for env/ps2_ (of ES and windows-stat measues)

In [ ]:
# mean across
ttrs_pos = cocoln2corrs_mesubj['None'].query('pval <= 0.05 and r > 0')
varnames = list(ttrs_pos.reset_index()['varn'].values)
#print(varnames )
print('For mean,then corr {} varnames are >0 correlated'.format(len(varnames) ) )

In [ ]:
# all trial together
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r', cols_addstat=['r'])
    return ttrs
m2vns={}

    # include groups is whether df that f receives as arg has grouping columns or not
ttrs = cocoln2corrs_per_subj.query('cocoln == "None"').\
    groupby(['varn','method'], observed=True).apply(f, include_groups=False)
print(getAddInfo())
#display(ttrs.query('pval <= 0.05 and ttstr == "r < 0"')\
#        [['pval','ttstr']])

ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0"')
#display(ttrs_pos[['pval','ttstr']])
for method in cocoln2corrs_per_subj.method.unique():
    varnames = ttrs_pos.loc[ttrs_pos.index.get_level_values('method') == method].reset_index()['varn'].values
    varnames = list(varnames)
    #varnames = list(ttrs_pos[(None,method)])
    #print(varnames )
    print(method, len(varnames), ' varnames are >0 correlated' )
    m2vns[method] = varnames

#print( ', '.join(varnames) )


#print ('Dif between methods ',set(m2vns['spearman']) ^ set(m2vns['shepherd']) )

In [ ]:
ttrs_pos = ttrs_pos.query('method == "spearman"')
ttrs_pos = ttrs_pos.sort_values(['pval'])
ttrs_pos[['pval','T','r_mean','r_std']]

In [ ]:
ttrs_pos = ttrs_pos.query('varn.str.contains("_invstd")').sort_values(['r_mean'], ascending=False)
ttrs_pos[['pval','T','r_mean','r_std']]

In [ ]:
#ttrs_pos.query('varn.str.endswith("std2")')

In [ ]:
#df_ = pd.DataFrame( {'varn':m2vns['spearman'] } )
#df_['']

# std2
# invstd 4-39
# abs invstd 2-39
# abs mav d var 4-39

m2vns['spearman']

In [ ]:
# plot ES vs windows stats
import warnings
varnames = list(ttrs_pos.reset_index()['varn'].values)
if len(varnames) <= 10:
    print(varnames)
    for env in ['stable','random']:
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore',category=FutureWarning)
            me = dfcs_fixhistlen.query('env == @env').\
                groupby(['subject','env'])[['err_sens']+varnames].mean().reset_index()
            fg = sns.pairplot(data=me, 
                     y_vars = ['err_sens'], hue='env',
                         x_vars=['err_sens'] + varnames)#,corner=1)

    me = dfcs_fixhistlen.\
        groupby(['subject'])[['err_sens']+varnames].mean().reset_index()
    fg = sns.pairplot(data=me, 
             y_vars = ['err_sens'], 
                 x_vars=['err_sens'] + varnames)#,corner=1)
else:
    print('Too long list of varnames ')

In [ ]:
gc.collect()

## corr within env (of ES and windows-stat measues)

In [ ]:
# mesubj (useless)
# it gives 0 because for random it does not make sense to average across participants
# because randomness is not consistent across them
env2varnames = {}
ttrs = cocoln2corrs_mesubj['env']
for env in ['stable','random']:
    ttrs_pos = ttrs.query('pval <= 0.05 and r > 0 and'
                         ' env == @env')
#display(ttrs_pos[['pval','ttstr']])
    varnames = list(ttrs_pos.reset_index()['varn'].values)
    print(env,len(varnames))
    env2varnames[env] = varnames
varnames_isec = set(env2varnames['stable']) & set(env2varnames['random'])
varnames_isec = list(varnames_isec)
print('random ', set(env2varnames['random']))
print('isec varnames',varnames_isec  )

print('For mean, stable&random corr {} varnames are >0 correlated'.format(len(varnames) ) )

In [ ]:
# Find with windows stats are positive bot for random AND for stable
from behav_proc import compare0
def f(df):
    if len(df) == 0:
        return None
    ttrs_ = compare0(df, 'r')
    return ttrs_
for method in cocoln2corrs_per_subj.method.unique():
    ttrs = cocoln2corrs_per_subj.query('cocoln == "env" and method == @method').\
        groupby(['varn','env'], observed=True).apply(f)
    print(getAddInfo())
    #display(ttrs.query('pval <= 0.05 and thr in @thrs')\
    #        [['pval','ttstr']])

    ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0"')
    print('All positive, both envs (separately)')
    #display(ttrs_pos[['pval','ttstr']])

    env2varnames = {}
    for env in ['stable','random']:
        ttrs_pos = ttrs.query('pval <= 0.05 and ttstr == "r > 0" and'
                             ' env == @env')
    #display(ttrs_pos[['pval','ttstr']])
        varnames = list(ttrs_pos.reset_index()['varn'].values)
        env2varnames[env] = varnames
    varnames_isec = set(env2varnames['stable']) & set(env2varnames['random'])
    varnames_isec = list(varnames_isec)
    print(method, 'isec varnames',varnames_isec  )
#print(varnames)


In [ ]:
# To be put into paper
mer = cocoln2corrs_per_subj.query('cocoln == "env"').\
    groupby(['varn','env'], observed=True)['r'].mean()

print(getAddInfo(),'\n')
r = ttrs.query('pval <= 0.05 and ttstr == "r > 0" '
    ' and varn.isin(@varnames_isec)').reset_index()[['varn','env','pval']].set_index(['varn','env'])
#d = r.T.to_dict()
#print(d)

for varn in varnames_isec:
    st = r.loc[(varn,'stable'),'pval']
    ra = r.loc[(varn,'random'),'pval']
    stabr = mer[(varn,'stable')]
    randr = mer[(varn,'random')]
    
    s = (f"{varn}: For random r>0 has p-val={ra:.2e} (mean r = {randr:.3f})"
        f" and for stable r>0 has p-val={st:.2e} (mean r = {stabr:.3f})")
    print(s)

In [ ]:
#me['env']

In [ ]:
# plot
h=4
varnames_toshow = varnames_isec[1:]
fg,df = relplot_multi(sep_ys_by = 'col', data=dfcs_fixhistlen, x='err_sens',ys=varnames_toshow, row='env',
                     height = h)
for ax in fg.axes.flatten():
    ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
    ax.set_title(ttl)

fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Pooled')
plt.tight_layout()

me = dfcs_fixhistlen.\
    groupby(['subject','env'])[['err_sens']+varnames_toshow].mean().reset_index()
fg,df = relplot_multi(sep_ys_by = 'col', data=me, x='err_sens',ys=varnames_toshow, row='env',
                     height = h)
for ax in fg.axes.flatten():
    ttl = ax.get_title().replace('__varname = ','').replace('env = ','')
    ax.set_title(ttl)
fg.refline(x=0)
fg.refline(y=0)
fg.fig.suptitle('Mean within subject')
plt.tight_layout()

In [ ]:
# fg = sns.relplot(data=dfcs_fixhistlen, col='subject',
#     y = 'err_sens', x=varnames_isec[0], row='env')

In [ ]:
from behav_proc import formatRecentStatVarnames
isec_nice = formatRecentStatVarnames(varnames_isec)
display(isec_nice)
print('; '.join(isec_nice))

In [ ]:
isec = set(env2varnames['stable']) & set(env2varnames['random'])
if len(isec):
    print('If we fix the history length first, then compute correlation '
      'within participant (separately for both environments)'
      ' and then choose variables for which r>0 stat significantly in both environments'
      ' simultaneously, we get ',', '.join(list(isec) ) )
else:
    print('We get nothing signif postive in both separately')

In [ ]:
#ttrs.query('varn == "error_pscadj_invstd10"')

In [ ]:
# plot
varnames = list(ttrs_pos.reset_index()['varn'].values)
print(varnames)
for env in ['stable','random']:
    plt.figure()
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore',category=FutureWarning)
        fg = sns.pairplot(data=dfcs.query('env == @env').\
            groupby(['subject','env'])[['err_sens']+env2varnames[env]].mean().reset_index(), 
            y_vars = ['err_sens'], hue='env',
            x_vars=['err_sens'] + env2varnames[env])#,corner=1)

In [ ]:
# You are an expert in python pandas and data science.
# I have a pandas dataframe with the following columns
# ttrs_pos.columns = ['alt', 'val1', 'ttstr','varn','env']
# 'ttstr' can take two possible string values '>0' and '<0'. 
# 'env' can take two string values 'stable' and 'random'. 
# 'varn' can take several possible string values.
# I want to select rows that have ttstr = '>0' for both stable and random values of 'env'.
# How to do it?